In [339]:
import zipline
# from zipline.examples import buyapple

%load_ext zipline

The zipline extension is already loaded. To reload it, use:
  %reload_ext zipline


In [341]:
%%zipline --start 2014-1-1 --end 2018-1-1 -o data/dma.pickle --capital-base 1000


from zipline.api import order, order_target, record, symbol, order_value, update_universe
import matplotlib.pyplot as plt
from yahoofinancials import YahooFinancials

# For this example, we're going to write a simple momentum script.  
# When the stock goes up quickly, we're going to buy; 
# when it goes down we're going to sell.  
# Hopefully we'll ride the waves.

# To run an algorithm in Quantopian, you need two functions: 
# initialize and handle_data.
def initialize(context):
    # The initialize function sets any data or variables that 
    # you'll use in your algorithm. 
    # For instance, you'll want to define the security 
    # (or securities) you want to backtest.  
    # You'll also want to define any parameters or values 
    # you're going to use later. 
    # It's only called once at the beginning of your algorithm.

    context.tickers = ["AAPL", "YELP", "YHOO", "MMM",  
                    "ABT", "AMD", "AMZN", "GOOG",  
                    "AXP", "AMGN", "BBY", "BLK",  
                    "CAT"]

    context.universe = [symbol(ticker) for ticker in context.tickers]  
    
    # In our example, we're looking at Apple.  
    # If you re-type this line you'll see 
    # the auto-complete that is available for security. 
    context.security = symbol('AAPL')

    # limit the number of companies returned in a fundamentals query
    context.limit = 10

def before_trading_start(context, data):
    # context.fundamentals = get_fundamentals(
    #     query(
    #         fundamentals.valuation_ratios.pb_ratio,
    #         fundamentals.valuation_ratios.pe_ratio,
    #     )
    #     .filter(
    #         fundamentals.valuation_ratios.pe_ratio < 14
    #     )
    #     .filter(
    #         fundamentals.valuation_ratios.pb_ratio < 2
    #     )
    #     .order_by(
    #         fundamentals.valuation.market_cap.desc()
    #     )
    #     .limit(context.limit)
    # )
    
    update_universe(context.fundamentals.columns.values)

# The handle_data function is where the real work is done.  
# This function is run either every minute 
# (in live trading and minute backtesting mode) 
# or every day (in daily backtesting mode).
def handle_data(context, data):
    # We've built a handful of useful data transforms for you to use,
    # such as moving average. 
    
    # To make market decisions, we're calculating the stock's 
    # moving average for the last 5 days and its current price. 
    average_price = data[context.security].mavg(5)
    current_price = data[context.security].price
    MA1 = data[context.security].mavg(50)
    MA2 = data[context.security].mavg(200)
    current_positions = context.portfolio.positions[context.security].amount
    
    # Another powerful built-in feature of the Quantopian backtester is the
    # portfolio object.  The portfolio object tracks your positions, cash,
    # cost basis of specific holdings, and more.  In this line, we calculate
    # the current amount of cash in our portfolio.   
    cash = context.portfolio.cash
    print("cash before trade %s" % (cash))

    for stock in data:
        current_position = context.portfolio.positions[stock].amount
        stock_price = data[stock].price
        plausible_investment = cash / context.limit
     
        share_amount = int(plausible_investment / stock_price)

        try:
                if stock_price < plausible_investment:
                    if current_position == 0:
                        if context.fundamentals[stock]['pe_ratio'] < 11:
                            order(stock, share_amount)
    
        except Exception as e:
            print(str(e))
    
    # Here is the meat of our algorithm.
    # If the current price is 1% above the 5-day average price 
    # AND we have enough cash, then we will order.
    # If the current price is below the average price, 
    # then we want to close our position to 0 shares.

    # if current_price > 1.01*average_price and cash > current_price:
        
    #     # Need to calculate how many shares we can buy
    #     number_of_shares = int(cash/current_price)
        
    #     # Place the buy order (positive means buy, negative means sell)
    #     order(context.security, +number_of_shares)
    #     print("Buying %s %s" % (number_of_shares, context.security.symbol))
        

    # elif current_price < average_price:
        
    #     # Sell all of our shares by setting the target position to zero
    #     order_target(context.security, 0)
    #     print("Selling %s" % (context.security.symbol))
    
    # You can use the record() method to track any custom signal. 
    # The record graph tracks up to five different variables. 
    # Here we record the Apple stock price.
    record(stock_price=data[context.security].price)

ImportError: cannot import name 'update_universe'

In [338]:
# https://au.investing.com/webmaster-tools/

import pandas

path = '/Users/d062864/Documents/01_code/pythonStarter/data'
fileName = 'asx_screener.csv'

df = pandas.read_csv(path+'/'+fileName)


df



,Name,Symbol,Last,Chg. %,Market Cap,Vol.,P/E Ratio,Price to Book (MRQ)
0,Westpac Banking,WBC,21.35,-4.04%,74.43B,13.48M,11.74,1.50
1,National Australia Bank,NAB,22.00,-5.54%,63.48B,15.21M,13.38,1.44
2,ANZ Banking Group,ANZ,22.14,-4.73%,62.76B,9.65M,10.91,1.31
3,Fortescue Metals,FMG,9.60,-4.48%,29.89B,25.60M,4.12,1.64
4,Scentre,SCG,3.26,-4.40%,17.05B,21.93M,15.35,0.76
5,Suncorp,SUN,11.20,-2.18%,14.54B,5.96M,14.31,1.28
6,Santos,STO,6.70,-2.62%,13.96B,8.30M,14.14,1.33
7,Dexus Property,DXS,12.50,-1.34%,13.71B,3.92M,8.94,1.29
8,Mirvac Group,MGR,3.15,-2.78%,12.39B,18.19M,12.64,1.24
9,AGL Energy,AGL,18.65,-1.64%,11.93B,2.60M,13.23,1.51
